# Import Packages and Classifiers

In [2]:
#!pip install --user scikit-learn

In [3]:
#!pip install --user matplotlib

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt


https://scikit-learn.org/stable/modules/cross_validation.html

# Import Data 

# First Dataset 
https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists?select=aug_train.csv

## Prep Data for Modeling 

## Decision Tree with Pruning 
https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html

https://www.analyticsvidhya.com/blog/2020/10/cost-complexity-pruning-decision-trees/

## Neural Network
https://scikit-learn.org/stable/modules/neural_networks_supervised.html

## Boosting 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

## Support Vector Machines 
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py

## K Nearest Neighbors
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html


# Second Dataset   

https://www.kaggle.com/arashnic/imbalanced-data-practice?select=aug_train.csv

## Prep Data for Modeling 

## Decision Tree with Pruning 

## Neural Network

## Boosting 

## Boosting 

## Support Vector Machines 